<a href="https://colab.research.google.com/github/naelmostafa/computer-vision/blob/main/Assignment-4/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies

## Download fiftyone

In [ ]:
!pip install fiftyone

## Imports

In [ ]:
import torch
import torchvision
from PIL import Image
from torchvision.transforms import functional as func
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Model intialization


*   Run on GPU if available
*   Load pre-trained Model
*   load model to GPU
*   Set to evalution mode

In [ ]:
# Run on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Faster R-CNN

In [ ]:
# Load pre-trained model
model_rcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn_v2(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
model_rcnn.to(device)
model_rcnn.eval()

print("Model Faster R-CNN")

Model Faster R-CNN


## FCOS: Fully Convolutional One-Stage Object Detection

In [ ]:
# Load pre-trained model
model_fcos = torchvision.models.detection.fcos_resnet50_fpn(weights=torchvision.models.detection.FCOS_ResNet50_FPN_Weights.DEFAULT)
model_fcos.to(device)
model_fcos.eval()

print("Model FOCS")

Model FOCS


## SSD

In [ ]:
model_ssd = torchvision.models.detection.ssd300_vgg16(weights=torchvision.models.detection.SSD300_VGG16_Weights.DEFAULT)
model_ssd.to(device)
model_ssd.eval()

print("Model SSD")

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth


  0%|          | 0.00/136M [00:00<?, ?B/s]

Model SSD


# Download COCO-2017 validation split and explore

## Download

In [ ]:
# Download test split
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
)

dataset.persistent = True

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Images already downloaded


INFO:fiftyone.utils.coco:Images already downloaded


Existing download of split 'validation' is sufficient


INFO:fiftyone.zoo.datasets:Existing download of split 'validation' is sufficient


Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'coco-2017-validation'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [ ]:
print(dataset)

Name:        coco-2017-validation
Media type:  image
Num samples: 5000
Persistent:  True
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    faster_rcnn:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    FCOS:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    Faster_RCNN:  fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_tp:      fiftyone.core.fields.IntField
    eval_fp:      fiftyone.core.fields.IntField
    eval_fn:      fiftyone.core.fields.IntField


## Explore

In [ ]:
print(dataset.first().ground_truth.detections[0])

<Detection: {
    'id': '63a055620b4e02ae95cf631d',
    'attributes': {},
    'tags': [],
    'label': 'potted plant',
    'bounding_box': [
        0.37028125,
        0.3345305164319249,
        0.038593749999999996,
        0.16314553990610328,
    ],
    'mask': None,
    'confidence': None,
    'index': None,
    'supercategory': 'furniture',
    'iscrowd': 0,
    'eval': 'fn',
    'eval_id': '',
}>


In [ ]:
session = fo.launch_app(dataset)

# Run test and add predections to dataset to visualize

In [ ]:
# Classes
classes = dataset.default_classes
print(len(classes))

91


In [ ]:
def predection_inference(model, dataset, label_name:str):
  classes = dataset.default_classes
  # Add predictions to samples
  with fo.ProgressBar() as pb:
      for sample in pb(dataset):
          # Load image
          image = Image.open(sample.filepath)
          image = func.to_tensor(image).to(device)
          c, h, w = image.shape
          
          # Perform inference
          preds = model([image])[0]
          labels = preds["labels"].cpu().detach().numpy()
          scores = preds["scores"].cpu().detach().numpy()
          boxes = preds["boxes"].cpu().detach().numpy()
          
          # Convert detections to FiftyOne format
          detections = []
          for label, score, box in zip(labels, scores, boxes):
              # Convert to [top-left-x, top-left-y, width, height]
              # in relative coordinates in [0, 1] x [0, 1]
              x1, y1, x2, y2 = box
              rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]

              detections.append(
                  fo.Detection(
                      label=classes[label],
                      bounding_box=rel_box,
                      confidence=score
                  )
              )
          
          # Save predictions to dataset
          sample[label_name] = fo.Detections(detections=detections)
          sample.save()

  print("Finished adding predictions")

In [ ]:
predection_inference(model_rcnn, dataset, "faster_rcnn")

 100% |███████████████| 5000/5000 [18.5m elapsed, 0s remaining, 5.0 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 5000/5000 [18.5m elapsed, 0s remaining, 5.0 samples/s]      


Finished adding predictions


In [ ]:
predection_inference(model_fcos, dataset, "fcos")

 100% |███████████████| 5000/5000 [14.2m elapsed, 0s remaining, 6.1 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 5000/5000 [14.2m elapsed, 0s remaining, 6.1 samples/s]      


Finished adding predictions


In [ ]:
predection_inference(model_ssd, dataset, "ssd")

 100% |███████████████| 5000/5000 [17.5m elapsed, 0s remaining, 6.0 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 5000/5000 [17.5m elapsed, 0s remaining, 6.0 samples/s]      


Finished adding predictions


# Evaluate

In [ ]:
# Only contains detections with confidence >= 0.75
high_conf_view_faster_rcnn = dataset.filter_labels("faster_rcnn", F("confidence") > 0.75, only_matches=False)
high_conf_view_fcos = dataset.filter_labels("fcos", F("confidence") > 0.75, only_matches=False)
high_conf_view_ssd = dataset.filter_labels("ssd", F("confidence") > 0.75, only_matches=False)

In [ ]:
results_f_rcnn = high_conf_view_faster_rcnn.evaluate_detections(
    "faster_rcnn",
    gt_field="ground_truth",
    eval_key="eval_f_rcnn",
    compute_mAP=True,
    iou=0.5
)

results_fcos = high_conf_view_fcos.evaluate_detections(
    "fcos",
    gt_field="ground_truth",
    eval_key="eval_fcos",
    compute_mAP=True,
    iou=0.5
)

results_ssd = high_conf_view_ssd.evaluate_detections(
    "ssd",
    gt_field="ground_truth",
    eval_key="eval_ssd",
    compute_mAP=True,
    iou=0.5
)

# Faster RCNN

In [ ]:
results_fcos.print_report()
print(f'mAP: {results_fcos.mAP()}')

In [ ]:
plot = results_fcos.plot_pr_curves()
plot.show()

In [ ]:
plot.freeze()

# FCOS results

*   Precision Recall f1-score support
*   mAP
*   pr curve



In [ ]:
results_fcos.print_report()
print(f'mAP: {results_fcos.mAP()}')

                precision    recall  f1-score   support

      airplane       0.15      0.97      0.26       143
         apple       0.11      0.77      0.19       361
      backpack       0.04      0.63      0.08       371
        banana       0.20      0.89      0.32       850
  baseball bat       0.08      0.62      0.14       146
baseball glove       0.10      0.74      0.18       148
          bear       0.15      0.93      0.26        71
           bed       0.05      0.90      0.10       163
         bench       0.03      0.61      0.07       427
       bicycle       0.09      0.77      0.16       343
          bird       0.27      0.83      0.41       898
          boat       0.14      0.85      0.24       618
          book       0.24      0.85      0.37      2735
        bottle       0.18      0.83      0.29      1388
          bowl       0.10      0.82      0.17       654
      broccoli       0.09      0.88      0.17       460
           bus       0.14      0.89      0.24  

In [ ]:
plot = results_fcos.plot_pr_curves()
plot.show()

# SSD

In [ ]:
results_ssd.print_report()
print(f'mAP: {results_ssd.mAP()}')

                precision    recall  f1-score   support

      airplane       0.93      0.49      0.64       143
         apple       0.67      0.03      0.05       239
      backpack       0.80      0.01      0.02       371
        banana       0.93      0.07      0.12       379
  baseball bat       0.80      0.08      0.15       146
baseball glove       0.88      0.05      0.09       148
          bear       0.98      0.61      0.75        71
           bed       0.94      0.37      0.54       163
         bench       0.89      0.11      0.20       413
       bicycle       0.92      0.14      0.24       316
          bird       0.98      0.14      0.25       440
          boat       0.91      0.07      0.14       430
          book       1.00      0.00      0.01      1161
        bottle       0.85      0.03      0.05      1025
          bowl       0.86      0.15      0.26       626
      broccoli       0.66      0.06      0.11       316
           bus       1.00      0.47      0.64  

In [ ]:
plot = results_ssd.plot_pr_curves()
plot.show()

# Explore hight Confidence view

## Faster RCNN

In [ ]:
# Show samples with most true positives
session.view = high_conf_view_faster_rcnn.sort_by("eval_f_rcnn_tp", reverse=True)

In [ ]:
# Show samples with most true positives
session.view = high_conf_view_faster_rcnn.sort_by("eval_f_rcnn_fp", reverse=True)

## FCOS

In [ ]:
# Show samples with most true positives
session.view = high_conf_view_fcos.sort_by("eval_fcos_tp", reverse=True)

In [ ]:
# Show samples with most true positives
session.view = high_conf_view_fcos.sort_by("eval_fcos_fp", reverse=True)

## SSD

In [ ]:
# Show samples with most true positives
session.view = high_conf_view_ssd.sort_by("eval_ssd_tp", reverse=True)

In [ ]:
# Show samples with most false positives
session.view = high_conf_view_ssd.sort_by("eval_ssd_fp", reverse=True)